# Linear Regression and Cross-Validation

Note: This is a revised assignment originally called "Homework 5" from Professor Katherine Kinnaird's Machine Learning Spring 2021 course (CSC 294).

Welcome to the first piece of machine learning for SDS 220! I'm excited to bridge the gap a little between intro stat and machine learning, because throughout both classes I've been thinking of ways concepts from one class could really complement concepts learned in the other. In this piece we will take a look at cross-validation and how it can be helpful for what we've been doing in our final projects. Cross-validation (colloquially called cross-val) will help us select the appropriate eplanatory variable or combination of explanatory variables for our linear regression models for predicting our outcome variable with the most accuracy. 

Throughout this piece I will be using my own intro stat final project group's data set, called "babies", which documents different physical and behavioral aspects of parents and the birth weights of their babies. For more information on what the variables and their respective values mean check out this [doc](https://docs.google.com/document/d/18R_gny_3b9zzn4zhHjbK09MwXSoT0NJsZNZCmuTu_po/edit). This data set is very specific and there are definitely some interesting aspects worth knowing about it (data collection, variables included, etc) that you can check out [here](https://docs.google.com/document/d/1QNnP3_OEKEgylCQNdS36R-yU5Y4hczN1FdcY5n0Lg-s/edit). We'll start by importing some packages for helper functions and tools we'll need later, similar to when we load packages using the library() function in R (for those unfamiliar with Python). 

In [8]:
# Import Block
import pandas as pd
import numpy as np
from sklearn import linear_model

In [9]:
# Data Import Block
babies_data = pd.read_csv("babies.csv")

We can take a peak at the data in a similar way to how we can view data in RStudio by simply calling the dataframe (however, in this editor this trick only works as long as this is the last thing we call in our code block):

In [10]:
babies_data

,id,pluralty,outcome,date,gestation,sex,wt,parity,race,age,...,drace,dage,ded,dht,dwt,marital,inc,smoke,time,number
0,15,5,1,1411,284,1,120,1,8,27,...,8,31,5,65,110,1,1,0,0,0
1,20,5,1,1499,282,1,113,2,0,33,...,0,38,5,70,148,1,4,0,0,0
2,58,5,1,1576,279,1,128,1,0,28,...,5,32,1,99,999,1,2,1,1,1
3,61,5,1,1504,999,1,123,2,0,36,...,3,43,4,68,197,1,8,3,5,5
4,72,5,1,1425,282,1,108,1,0,23,...,0,24,5,99,999,1,1,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,9153,5,1,1672,275,1,113,0,0,27,...,0,32,4,72,190,1,4,0,0,0
1232,9163,5,1,1712,265,1,128,1,0,24,...,0,24,5,73,170,1,3,0,0,0
1233,9213,5,1,1672,291,1,130,4,1,30,...,2,30,5,70,180,1,3,1,1,2
1234,9229,5,1,1680,281,1,125,0,0,21,...,0,27,5,71,165,1,1,0,0,0


This data set requires a bit of wrangling before we work with it:

In [11]:
# Filter for only the columns we want
bb_data = babies_data[['gestation', 'race', 'age', 'smoke', 'wt']]

# Filter for unknowns
bb_data = bb_data[bb_data['gestation'] != 999]
bb_data = bb_data[bb_data['wt'] != 999]
bb_data = bb_data[bb_data['race'] != 99]
bb_data = bb_data[bb_data['age'] != 99]
bb_data = bb_data[bb_data['smoke'] != 9]

# race needs some extra pre-processing
bb_data["race"].replace({1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 1, 7: 2, 8: 4, 9: 5, 10: 6}, inplace=True)

bb_data.to_csv("babies_wrangled.csv", index = False)

In [12]:
babies_wrangled = pd.read_csv("babies_wrangled.csv")

In [13]:
# Extra function we will use later for cross-val

def compute_mse(truth_vec, predict_vec):
    return np.mean((truth_vec - predict_vec)**2)

## Pre-processing the data

Many people are surprised to know that linear regression is actually a form of machine learning! So we've all basically been doing machine learning for years and we didn't even know it, which is pretty cool when you think about it. 

The package we'll be using for our linear regression implementation, Numpy, can only fit the linear regression model to it's own data type called a Numpy array. Numpy arrays cannot contain non-numeric inputs, which means that if our data has categorical variables, we'll need to convert them to numbers. Luckily, the babies dataset already uses numbers to indicate the values of categorical variables. However, if you wanted to perform cross-val for linear regression on your own data set and your data has categorical variables with string values, the function described below will be helpful for you.

For example, the `smoke` variable in our data set uses numbers to indicate
cartegories as follows: 0 = never smoked, 1 = smokes now, 2 = smoked until pregnancy
3 = smoked once, but not now, and 9 = unknown.

For this problem, I've created a function `data_wrangle`. The function (implemented below) takes the csv filename for the data set you're using and a list of columns from the dataset 
that need to be converted to numerical values. The behavior of the function is summarized in the following steps:

1. Import the data with `pandas` read_csv() function
2. For each column listed as an input (ie. those with non-numerical 
   data):
   * Determine the unique values in that column
   * Convert the unique values in into unique numbers. 
3. Get the data's column names from 
   the observation rows.  
4. Convert the data to a numpy array
5. Return all of the column names in 
   order and the converted dataset as a tuple. 
   
If you're unsure what the function is doing at any point, you can add print statements to print out variable values (syntax: "print(data)"). For those that are not comfortable with certain coding elements or conventions (like for-loops, function-writing, and methods), unfortunately these elements will not be explained here, but if you'd like to learn more about them I'd recommend taking CSC 111: Intro to Computer Science.

In [14]:
def data_wrangle(dataset_file, lst):
    
    # Import data with pandas
    data = pd.read_csv(dataset_file)
    
    for column in lst:
        # tip retrieved from https://cmdlinetips.com/2018/01/
        # how-to-get-unique-values-from-a-column-in-pandas-data-frame/
        unique_values = data[column].unique()
        
        numeric_val = 0
        col_names = []
        
        for value in unique_values:
            
            # For clarity
            print(value , "in column" , column , "is now" , numeric_val)
            
            data = data.replace(value, numeric_val)
            numeric_val += 1
            
            # Extract the data's column names from the observation rows
            col_names = data.loc[data[column] == value].columns
            
    if lst == []:
        col_names = data.columns
            
    return np.array(col_names), data.to_numpy()

## Implement k-fold Cross-Validation

Okay, I've been saying cross-validation a lot, but what actually is it? For this piece, we'll be using k-fold cross-validation for model selection, so I'll be focusing specifically on this type of cross-val. To understand cross-val, we'll first have to define some other concepts.

In machine learning, we have this notion of train vs test. When we want to test the performance of a model, we'll usually split our data into training data and testing data. We train the model on the training data by fitting it to only that data, then test its performance on new data it hasn't seen before by asking it to predict values for this new data. But, when we're splitting data into training and testing data we run into some practical problems. What's the appropriate split? 50-50? Or something more extreme like 95-5? The more training data our model sees, the more accurate it will become. But if we leave too little data in either group, then we have a small sample size where we risk getting values that are not representative of the population. But what if we could overcome these issues by including all the data in our training and testing groups? This is what cross-val aims to to do.

In k-fold cross-val, we divide our data into a certain number "k" of groups or "folds", and we let each group/fold take a turn being the test set, while we designate every other fold to be in the training set for a specific iteration. For a more concrete example of what this actually looks like, consider the following scenario:

I split my data into 10 groups. I then select the first group and designate it as the testing set. The rest of the 9 groups are lumped together to become the training data set. I train my model (in our case our linear regression model) on the the data in the training set (groups 2-10), and then I ask it to predict the values of the outcome variable based on the values of the explanatory variables in the test set. After this, I compare the true values of the outcome variable in the test set to the values that my model predicted, and I compute the mean-squared error of the truth values vs the predicted values (this is basically taking the average of the squared values of the residuals). This is one iteration through the data. For a complete k-fold cross-val, we repeat the above process for the second group, the third group, and so on until every group has had a turn being the test set. We then compute the mean of all the mean-squared errors we computed for each iteration to get the k-fold cross-val error.

Still feeling like the concept is a little too abstract? Let's take a look at cross-val in action and see if we can clear things up.

For this next part I've created a function `kfold_CV` (in `hw5.py`) that implements k-fold 
cross-validation. 
The inputs, in order, are:
1. The full dataset (as a numpy array)
2. The header list (as in the list of variable/column names, in the 
   the order that they appear in the dataset)
3. Input variables
4. Output variables
5. Value for _k_

Using these inputs, `kfold_CV` is able to select the input and output columns from the data set, divide the data into k folds, and then iteratively choose one fold to be the test set for k rounds of training and testing. The linear regression model is trained on 100-100/k% of the 
data each iteration, with the input and output variables, and then is asked to 
predict the values of ouput variable based on the input variable values in the
test set. The error is then computed from the true and predicted values for each iteration,
with the final cross-val error being the mean of all these errors.

In [15]:
def kfold_CV(data, col_names, inputs, output, k):
    # Shuffle data so cross-val error is not coincidentally zero
    np.random.shuffle(data)
    
    ### Note: does not always split data evenly depending on number of rows and value of k
    if data.shape[0]%k != 0:
        maximum = data.shape[0]%k
    else:
        maximum = -1
    
    # If inputs or output is given as a single variable string convert to list
    if type(inputs) != list:
        inputs = [inputs]
        
    if type(output) != list:
        output = [output]
    
    # Convert columns to np array if they are passed as a list for np.argwhere to function properly   
    if type(col_names) == list:
        col_names = np.array(col_names)
    
    input_col_inds = []
    output_col_inds = []
    
    # Find numerical column index for input and output variables using list of ordered columns
    for i in inputs:
        input_col_inds.append(np.argwhere(col_names == str(i))[0][0])
    for p in output: 
        output_col_inds.append(np.argwhere(col_names == str(p))[0][0])
        
    num_rows = data.shape[0]
    
    start = 0
    end = num_rows//k
    count = 0
    test_errors = []

    # Loop over all folds in the data set, letting each act as the test set
    for fold in range(k):
        
        # Split data into train and test
        test_data = data[start:end, :]
        train_indices = list(set(range(num_rows)).difference(list(set(range(start, end)))))
        train_data = data[train_indices, :]
        
        # Create and train a model
        lm = linear_model.LinearRegression()
        mod = lm.fit(train_data[:, input_col_inds], train_data[:, output_col_inds])
    
        # Compute the testing error and add it to the list of testing errors
        test_preds = mod.predict(test_data[:, input_col_inds])
        test_error = compute_mse(test_preds, test_data[:, output_col_inds])
        test_errors.append(test_error)
                             
        start = start + num_rows//k
        end = start + num_rows//k
        count += 1
        
        if count <= maximum:
            start += 1
            end += 1
        
    # Compute the cross-val error
    return np.mean(test_errors)

## Using 10-fold Cross-Validation to Pick Explanatory Variables

Now that we've defined our cross-val function, we can use it to select the best combination of explanatory variables for our linear regression model. Below I've tried all combinations of one, two, three, and finally four variables to see which one yields the lowest cross-val error for our data set. I decided to use 10-fold cross-validation because a 90% train - 10% test split is usually considered a pretty good train-test split.

In [16]:
data_np = data_wrangle("babies_wrangled.csv", [])

In [17]:
# Transform data
data_np = data_wrangle("babies_wrangled.csv", [])

all_errors = []

## One-variable combinations

# Input: gestation
g_error = kfold_CV(data_np[1], data_np[0], ["gestation"], ["wt"], 10)
all_errors.append(g_error)
print("10-fold cross-val for gestation vs weight is ", g_error)

# Input: race
r_error = kfold_CV(data_np[1], data_np[0], ["race"], ["wt"], 10)
all_errors.append(r_error)
print("10-fold cross-val for race vs weight is ", r_error)

# Input: age
a_error = kfold_CV(data_np[1], data_np[0], ["age"], ["wt"], 10)
all_errors.append(a_error)
print("10-fold cross-val for age vs weight is ", a_error)

# Input: smoke
s_error = kfold_CV(data_np[1], data_np[0], ["smoke"], ["wt"], 10)
all_errors.append(s_error)
print("10-fold cross-val for smoke vs weight is ", s_error)

10-fold cross-val for gestation vs weight is  278.6868521406365
10-fold cross-val for race vs weight is  324.17065825257055
10-fold cross-val for age vs weight is  332.8886852969867
10-fold cross-val for smoke vs weight is  332.8530570981478


In [18]:
## Two-variable combinations

# Input: gestation and race
gr_error = kfold_CV(data_np[1], data_np[0], ["gestation", "race"], ["wt"], 10)
all_errors.append(gr_error)
print("10-fold cross-val for gestation and race vs weight is ", gr_error)

# Input: gestation and age
ga_error = kfold_CV(data_np[1], data_np[0], ["gestation", "age"], ["wt"], 10)
all_errors.append(ga_error)
print("10-fold cross-val for gestation and age vs weight is ", ga_error)

# Input: gestation and smoke
gs_error = kfold_CV(data_np[1], data_np[0], ["gestation", "smoke"], ["wt"], 10)
all_errors.append(gs_error)
print("10-fold cross-val for gestation and smoke vs weight is ", gs_error)

# Input: race and age
ra_error = kfold_CV(data_np[1], data_np[0], ["race", "age"], ["wt"], 10)
all_errors.append(ra_error)
print("10-fold cross-val for race and age vs weight is ", ra_error)

# Input: race and smoke
rs_error = kfold_CV(data_np[1], data_np[0], ["race", "smoke"], ["wt"], 10)
all_errors.append(rs_error)
print("10-fold cross-val for race and smoke vs weight is ", rs_error)

# Input: age and smoke
as_error = kfold_CV(data_np[1], data_np[0], ["age", "smoke"], ["wt"], 10)
all_errors.append(as_error)
print("10-fold cross-val for age and smoke vs weight is ", as_error)

10-fold cross-val for gestation and race vs weight is  275.99051625522696
10-fold cross-val for gestation and age vs weight is  278.22216196184667
10-fold cross-val for gestation and smoke vs weight is  279.2821110010082
10-fold cross-val for race and age vs weight is  324.3208239907482
10-fold cross-val for race and smoke vs weight is  324.54081690217293
10-fold cross-val for age and smoke vs weight is  333.62879561646594


In [19]:
## Three-variable combinations

# Input: gestation, race, and age
gra_error = kfold_CV(data_np[1], data_np[0], ["gestation", "race", "age"], ["wt"], 10)
all_errors.append(gra_error)
print("10-fold cross-val for gestation, race, and age vs weight is ", gra_error)

# Input: gestation, race, and smoke
grs_error = kfold_CV(data_np[1], data_np[0], ["gestation", "race", "smoke"], ["wt"], 10)
all_errors.append(grs_error)
print("10-fold cross-val for gestation, race, and smoke vs weight is ", grs_error)

# Input: gestation, age, and smoke
gas_error = kfold_CV(data_np[1], data_np[0], ["gestation", "age", "smoke"], ["wt"], 10)
all_errors.append(gas_error)
print("10-fold cross-val for gestation, age, and smoke vs weight is ", gas_error)

# Input: race, age, and smoke
ras_error = kfold_CV(data_np[1], data_np[0], ["race", "age", "smoke"], ["wt"], 10)
all_errors.append(ras_error)
print("10-fold cross-val for race, age, and smoke vs weight is ", ras_error)


10-fold cross-val for gestation, race, and age vs weight is  274.5246321507387
10-fold cross-val for gestation, race, and smoke vs weight is  276.2099936870532
10-fold cross-val for gestation, age, and smoke vs weight is  278.5582633434022
10-fold cross-val for race, age, and smoke vs weight is  323.9854941339939


In [20]:
## Four-Variable Combination

# Input: gestation, race, age, and smoke
gras_error = kfold_CV(data_np[1], data_np[0], ["gestation", "race", "age", "smoke"], ["wt"], 10)
all_errors.append(gras_error)
print("10-fold cross-val for gestation, race, age, and smoke vs weight is ", gras_error)


10-fold cross-val for gestation, race, age, and smoke vs weight is  276.2084872717013


In [21]:
print("Minimum error is", min(all_errors), 
      ", which is the 10-fold cross-val error for \ngestation, race, and age as input variables")

Minimum error is 274.5246321507387 , which is the 10-fold cross-val error for 
gestation, race, and age as input variables


We can now see that the best combination of expalnatory variables for predicting birth weight with a linear regression model is the length of gestation, the mother's race, and the mother's age, because this combination yielded the lowest cross-val error. This means that this combination predicted infant birth weight for the test set with the least deviation from the true birth weight values across 10 different iterations of splitting 90% of the data into train, and 10% to test, out of all the possible explanatory variable combinations.

## Determining the full model

Now that we've selected our optimal combination of variables, we can fit our model to all the data instead of training it on only a fraction of the data. When we use all of the data instead of a fraction of it, we have more points to fit our model to, making our model as accurate a representation of the data as possible

In [22]:
# Set up linear regression
lm2 = linear_model.LinearRegression()

# Fit the linear regression to the data
model2 = lm2.fit(data_np[1][:, 0:3], data_np[1][:, 4])

# Extract the coefficients
m = model2.coef_
b = model2.intercept_
print("slope coefficients are", m[0], m[1], "and", m[2], "\nintercept is", b)

slope coefficients are 0.4446675816423984 -1.5223872112036072 and 0.17934673343344268 
intercept is -8.413038906983132


So, we can write the equation for the full model as follows:

\begin{equation}
\widehat{birth\: weight} = .44 \cdot gestation -1.52 \cdot race + .179 \cdot age - 8.41 \: ounces
\end{equation}

## Conclusion

Now we know how to select the appropriate explanatory variable combination for our outcome variable using cross-validation! You might be wondering at this point: wait, is 275 a large cross-val error to obtain in this scenario? How can we visualize the accuracy of our model in four dimensions? These are of course good questions, but they are unfortunately outside the scope of this piece. If we were trying to select the best variable combinations for models we know how to visualize, we might only use cross-val with one or two explanatory variables for our linear regression. We might also dimension reduce our data to better visualize it, which I will talk about next in the piece on PCA.